_Note: working in my spring-env environment, see requirements.txt_

Notebook is based on original final project script: prepare_household_energy_data.R to read in raw csv and output cleaned data "daily_energy_all_hholds.csv"

# Setup

In [1]:
import pandas as pd
import numpy as np

# Clean data

In [2]:
# read in raw csv, sorted by Date column (ascending order, numerical!)
raw_data = pd.read_csv("raw_data/london_energy.csv").sort_values(by=['Date']).copy()

In [3]:
raw_data

,LCLid,Date,KWH
115432,MAC000154,2011-11-23,5.798
110458,MAC000148,2011-11-23,1.283
109206,MAC000146,2011-11-23,5.619
111287,MAC000149,2011-11-23,2.287
112945,MAC000151,2011-11-23,3.273
...,...,...,...
1079411,MAC001613,2014-02-28,0.214
2459442,MAC004005,2014-02-28,0.042
2460045,MAC004006,2014-02-28,0.228
1078766,MAC001612,2014-02-28,0.334


In [4]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3510433 entries, 115432 to 3510432
Data columns (total 3 columns):
 #   Column  Dtype  
---  ------  -----  
 0   LCLid   object 
 1   Date    object 
 2   KWH     float64
dtypes: float64(1), object(2)
memory usage: 107.1+ MB


Next step in plan is to *clean column names (all to lowercase, snake case)* -- and name processed data as df:

In [5]:
dfclean = raw_data.rename(columns={"LCLid": "lc_lid", "Date": "date", "KWH": "kwh"}).copy()

In [6]:
dfclean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3510433 entries, 115432 to 3510432
Data columns (total 3 columns):
 #   Column  Dtype  
---  ------  -----  
 0   lc_lid  object 
 1   date    object 
 2   kwh     float64
dtypes: float64(1), object(2)
memory usage: 107.1+ MB


## Wrangle date column

Need to use datetime data for next step, here it is encoded as "object", which means it is string or mixed number/string. Needs to be datetime64 for pandas

In [7]:
dfclean['date']= pd.to_datetime(dfclean['date'])
#raw_data['Date'].astype("datetime64")

In [8]:
dfclean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3510433 entries, 115432 to 3510432
Data columns (total 3 columns):
 #   Column  Dtype         
---  ------  -----         
 0   lc_lid  object        
 1   date    datetime64[ns]
 2   kwh     float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 107.1+ MB


### trim data for dates: 2011-12-01 to 2014-02-27



In [9]:
### trim data for dates: 2011-12-01 to 2014-02-27
df = dfclean[(dfclean['date'] >= '2011-12-01') & (dfclean['date'] <= '2014-02-27')].copy()

Trimmed from 3510433 entries (dfclean) to 3505100 (df) - check what has been removed

In [10]:
min(dfclean['date'])

Timestamp('2011-11-23 00:00:00')

In [11]:
max(dfclean['date'])

Timestamp('2014-02-28 00:00:00')

In [12]:
min(df['date'])

Timestamp('2011-12-01 00:00:00')

In [13]:
max(df['date'])

Timestamp('2014-02-27 00:00:00')

In [14]:
dfclean.shape[0]-df.shape[0]

5333

trim removes 5333 rows. Note: .shape is the df structure, [0] is for rows, [1] is for columns.

Can also check the complement:

In [15]:
# use ~ as NOT IN (set membership) clause - specific to pandas
dfbin = dfclean[~((dfclean['date'] >= '2011-12-01') & (dfclean['date'] <= '2014-02-27'))].copy()
dfbin['date'].value_counts()

date
2014-02-28    4987
2011-11-30      76
2011-11-29      63
2011-11-28      55
2011-11-26      41
2011-11-27      41
2011-11-25      32
2011-11-24      25
2011-11-23      13
Name: count, dtype: int64

This looks correct.

In [20]:
df # see current df

,lc_lid,date,kwh
164829,MAC000216,2011-12-01,18.689
142390,MAC000188,2011-12-01,3.854
115969,MAC000155,2011-12-01,11.794
136082,MAC000180,2011-12-01,7.821
156610,MAC000206,2011-12-01,10.399
...,...,...,...
3472396,MAC005503,2014-02-27,4.611
588547,MAC000832,2014-02-27,10.749
2449287,MAC003988,2014-02-27,3.171
3159766,MAC005045,2014-02-27,4.720


### add columns for month, season, quarter, weekday type (so can filter for Summer and Winter 2013)

In [17]:
# create month, weekday, quarter columns from date
#dftrim['month'] = pd.DatetimeIndex(dftrim['date']).month #month as a number
#dftrim['wday'] = pd.DatetimeIndex(dftrim['date']).weekday #The day of the week with Monday=0, Sunday=6.
#dftrim['qtr'] = pd.DatetimeIndex(dftrim['date']).quarter

Want to create season where month 12-2 is winter

DatetimeIndex.quarter uses month 1-3 as quarter 1

could manually create conditional column (where month = 12,1,2; winter)
or could make new col as date +1 month and generate quarter from this

ultimately want one col with yearseason by which to filter

In [18]:
#df[(df.month == 12)] 

### write to new csv